In [15]:
import torch

# Dense Layer

In [16]:
# Define Dense Layer class
class DenseLayer:
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases with small random values
        self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
        self.biases = torch.zeros((1, n_neurons))

    def forward(self, inputs):
        # Perform forward pass through the layer
        self.output = torch.matmul(inputs, self.weights) + self.biases


# Activation Functions

In [17]:

# Define Activation classes
class ActivationReLU:
    def forward(self, inputs):
        # Apply ReLU activation function element-wise
        self.output = torch.max(torch.tensor(0), inputs)

class ActivationSigmoid:
    def forward(self, inputs):
        # Apply Sigmoid activation function element-wise
        self.output = 1 / (1 + torch.exp(inputs * -1))

class ActivationSoftmax:
    def forward(self, inputs):
        # Apply Softmax activation function for probability distribution
        exp_values = torch.exp(inputs - torch.max(inputs, axis=1, keepdim=True).values)
        probabilities = exp_values / torch.sum(exp_values, axis=1, keepdim=True)
        self.output = probabilities

class LinearActivation:
    def forward(self, x):
        # Linear activation, no transformation
        self.output = x


 # Neural Network Class with Training Functionality

In [18]:
# Define the Neural Network class
class NeuralNetwork:
    def __init__(self):
        # Initialize the neural network with a hidden layer, activation functions, and an output layer
        self.hidden_layer = DenseLayer(2, 4)
        self.activation1 = ActivationSigmoid()
        self.output_layer = DenseLayer(4, 2)
        self.activation2 = LinearActivation()

    def forward_pass(self, X):
        # Perform the forward pass through the neural network
        self.hidden_layer.forward(X)
        self.activation1.forward(self.hidden_layer.output)
        self.output_layer.forward(self.activation1.output)
        self.activation2.forward(self.output_layer.output)
        return self.activation2.output

    def backward_pass(self, y_pred, y_true):
        # Set the learning rate
        lr = torch.tensor(0.01)

        # Calculate gradients and update weights and biases for the output layer
        output_layer_gradients = y_pred - y_true
        self.output_layer.weights -= lr * torch.mm(self.activation1.output.t(), output_layer_gradients)
        self.output_layer.biases -= lr * torch.sum(output_layer_gradients, dim=0, keepdim=True)

        # Calculate gradients and update weights and biases for the hidden layer
        hidden_layer_gradients = torch.mm(output_layer_gradients, self.output_layer.weights.t()) * \
                                self.activation1.output * (1 - self.activation1.output)
        self.hidden_layer.weights -= lr * torch.mm(X.view(1, -1).t(), hidden_layer_gradients)
        self.hidden_layer.biases -= lr * torch.sum(hidden_layer_gradients, dim=0, keepdim=True)

    def train(self, X, y, loss_threshold=0.00000001):
        # Train the neural network using forward and backward passes until the loss reaches the threshold
        y_pred = self.forward_pass(X)
        err = self.error_calculation(y, y_pred)
        print("Initial loss:", err)
        print("Initial prediction:", y_pred)

        while err > loss_threshold:
            self.backward_pass(y_pred, y)
            y_pred = self.forward_pass(X)
            err = self.error_calculation(y, y_pred)

        print("Final loss:", err)
        print("Final prediction:", y_pred)
        print("Target value:", y)

    def error_calculation(self, y_true, y_pred):
        # Calculate the mean squared error between true and predicted values
        return torch.mean(0.5 * (y_true - y_pred) ** 2)


#  Main Code - Neural Network Training

In [19]:
# Main code for training a neural network

if __name__ == "__main__":
    # Define input data and target values
    X = torch.tensor([0.1, 0.5])
    y = torch.tensor([0.05, 0.95])

    # Create an instance of the NeuralNetwork class
    neural_network = NeuralNetwork()

    # Train the neural network using the specified input and target values
    neural_network.train(X, y)

Initial loss: tensor(0.2216)
Initial prediction: tensor([[0.0152, 0.0092]])
Final loss: tensor(9.8431e-09)
Final prediction: tensor([[0.0500, 0.9498]])
Target value: tensor([0.0500, 0.9500])
